# Particle Fluid

Modeling fluid motion by simulating a set of particles. 

Credit to Professor Bo Zhu, on whose Dartmouth COSC89 course notes the following code is based

### Structure: 
Give each particle an index
each particle has a position (2D vector), velocity (2D vector), mass (scalar), density (scalar), pressure (scalar)
Have a global constant h (kernel radius)

Have kernels to calculate w_spiky, gradient_w_spiky, laplacian_w_spiky
Where w_spiky takes two indices (which it uses to calculate the radius r)
grad and laplacian take just one index (which it uses to find the velocity of that index)

Have kernels to calculate w_vis, gradient_w_vis, laplacian_w_vis
Same inputs as for the corresponding spiky kernels

Spatial Hashing: 
Keep a dictionary hashing a vec2 -> list of indices
Iterate over all points and add them to the array that corresponds 

Have each particle keep track of its neighbors with an array of arrays

Use explicit time integration to advance velocity and position for each time step

Have methods to update:
Neighbors
Density
Pressure
Pressure Force
Viscosity Force
Body Force
Boundary Collision Force
Temperature and Buoyancy (?)
